In [7]:
import torch
# import torchtext
# from torchtext.data.utils import get_tokenizer
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import DataLoader

In [8]:
torch.__version__

'2.1.1'

In [9]:
torchtext.__version__

NameError: name 'torchtext' is not defined

# TorchText文本分类数据集

In [ ]:
train_iter, test_iter = torchtext.datasets.IMDB()

In [ ]:
print(next(train_iter))

('neg', 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far bet

In [ ]:
tokenizer = get_tokenizer('basic_english')             # 初始化分词工具
print(tokenizer('This is a book about PyTorch.'))      # 在英文语句上调用并打印分词结果

['this', 'is', 'a', 'book', 'about', 'pytorch', '.']


In [ ]:
# train_iter, test_iter = torchtext.datasets.IMDB()
train_data, test_data = list(train_iter), list(test_iter)

In [ ]:
train_data[0]

('neg',
 '"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn\'t matter what one\'s political views are because this film can hardly be taken seriously on any level. As for the claim that frontal male nudity is an automatic NC-17, that isn\'t true. I\'ve seen R-rated films with male nudity. Granted, they only offer some fleeting views, but where are the R-rated films with gaping vulvas and flapping labia? Nowhere, because they don\'t exist. The same goes for those crappy cable shows: schlongs swinging in the breeze but not a clitoris in sight. And those pretentious indie movies like The Brown Bunny, in which we\'re treated to the site of Vincent Gallo\'s throbbing johnson, but not a trace of pink visible on Chloe Sevigny. Before crying (or implying) "double-standard" in matters of nudity, the mentally obtuse should take into account one unavoidably obvious anatomical difference between men and women: there are no genitals on display when actresses appears nude, 

In [ ]:
all_classes = set([label for (label, text) in train_data])
num_class = len(all_classes)

In [ ]:
all_classes

{'neg', 'pos'}

In [ ]:
from torchtext.data.utils import get_tokenizer            # 分词工具
from torchtext.vocab import build_vocab_from_iterator     # 创建词表工具

In [ ]:
tokenizer = get_tokenizer('basic_english')      # 分词工具做初始化

In [ ]:
def yield_tokens(data):
    for _, text in data:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_data), specials=["<pad>", "<unk>"], min_freq=5)

In [ ]:
vocab["<pad>"]

0

In [ ]:
vocab["<unk>"]

1

In [ ]:
vocab.set_default_index(vocab["<unk>"])

In [ ]:
token = tokenizer('I will try my best to make all people i like better.')
print(token)

['i', 'will', 'try', 'my', 'best', 'to', 'make', 'all', 'people', 'i', 'like', 'better', '.']


In [ ]:
vocab(token)

[13, 86, 353, 64, 124, 8, 105, 37, 85, 13, 45, 133, 3]

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x == 'pos')

In [ ]:
text_pipeline('I will try my best to make all people i like better.')

[13, 86, 353, 64, 124, 8, 105, 37, 85, 13, 45, 133, 3]

In [ ]:
label_pipeline('pos')

1

In [ ]:
text_list_o = []
text_list_o.append(torch.tensor(text_pipeline('fuck u,LYB,NMSL'),dtype=torch.int64))
text_list_o.append(torch.tensor(text_pipeline('cubit so dump.'),dtype=torch.int64))

text_list_1 = torch.cat(text_list_o)
text_list_2 = torch.nn.utils.rnn.pad_sequence(text_list_o)
# 注意是按列看的
print(text_list_o)

print(text_list_1)
print(text_list_2)


[tensor([   1, 1228,    4,    1,    4,    1]), tensor([   1,   44, 6575,    3])]
tensor([   1, 1228,    4,    1,    4,    1,    1,   44, 6575,    3])
tensor([[   1,    1],
        [1228,   44],
        [   4, 6575],
        [   1,    3],
        [   4,    0],
        [   1,    0]])


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# embeddingbag 层输入 拼接好的batch句子 和 offset（每个句子起始的位置）但是RNN不需要offset
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        precess_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(precess_text)
    label_list = torch.tensor(label_list)
    # 此时，text_list的长度不同 通过torch.nn.utils.rnn.pad_sequence 按照序列中最长的序列进行填充
    text_list = torch.nn.utils.rnn.pad_sequence(text_list)
    return label_list.to(device), text_list.to(device)

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=False, collate_fn=collate_batch)

In [ ]:
for i, (l, b) in enumerate(train_dataloader):
    print(l.size(), b.size())
    if i>9:
        break

torch.Size([16]) torch.Size([535, 16])
torch.Size([16]) torch.Size([763, 16])
torch.Size([16]) torch.Size([535, 16])
torch.Size([16]) torch.Size([339, 16])
torch.Size([16]) torch.Size([1127, 16])
torch.Size([16]) torch.Size([372, 16])
torch.Size([16]) torch.Size([1169, 16])
torch.Size([16]) torch.Size([598, 16])
torch.Size([16]) torch.Size([727, 16])
torch.Size([16]) torch.Size([1049, 16])
torch.Size([16]) torch.Size([862, 16])


# 创建模型

In [ ]:
vocab_size = len(vocab)

In [ ]:
h1,c1 = nn.LSTM(input,(h0,c0))

In [ ]:
class LSTM_net():
    def __init__(self, input_seq_len, hidden):
        super().__init__()
        self.rnn = nn.LSTMCell(input_seq_len,hidden)
    def forward(self, inputs):
        bz = inputs.shape[1]
        ht = torch.zeros(bz, hidden).to(device)
        ct = torch.zeros(bz, hidden).to(device)
        for word in inputs:
            ht,ct = self.rnn(word,(ht,ct))
        return ht, ct


In [ ]:
class RNN_Net(nn.Module):
    def __init__(self, vocab_size, embeding_dim, hidden_size):
        super(RNN_Net, self).__init__()
        self.em = nn.Embedding(vocab_size, embeding_dim)   
        # embedding 会依照vocab的大小对输入的单词进行编码，到embeding_dim这样的大小
        self.rnn = nn.LSTM(embeding_dim, hidden_size)
        
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, inputs):
        x = self.em(inputs)
        x, _ = self.rnn(x)
#        print(x.size(), o.size())
        x = F.relu(self.fc1(x[-1]))
        x = self.fc2(x)
        return x

In [ ]:
embeding_dim = 256
hidden_size = 128
model = RNN_Net(vocab_size, embeding_dim, hidden_size).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
from torch.optim import lr_scheduler
optimizer = torch.optim.Adam(model.parameters(), betas=(0.5, 0.5), lr=0.01)
# betas=(0.5, 0.5) 一阶矩估计和二阶矩估计均为0.5
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [ ]:
for label, text in test_dataloader:
    print(label,text)
    break

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) tensor([[  13,  295,   99,  ...,   13,    6,  157],
        [ 125,    2,    6,  ...,  220,  589,    4],
        [1051,  706,  483,  ...,    2,    7,   13],
        ...,
        [   0,    0,    0,  ...,   19,    0,    0],
        [   0,    0,    0,  ...,   15,    0,    0],
        [   0,    0,    0,  ...,    3,    0,    0]])


In [ ]:
def train(dataloader):
    total_acc, total_count, total_loss, = 0, 0, 0
    model.train()
    for label, text in dataloader:
        predicted_label = model(text)
        loss = loss_fn(predicted_label, label)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()*label.size(0)
    return total_loss/total_count, total_acc/total_count

In [ ]:
def test(dataloader):
    model.eval()
    total_acc, total_count, total_loss, = 0, 0, 0

    with torch.no_grad():
        for label, text in dataloader:
            predicted_label = model(text)
            loss = loss_fn(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()*label.size(0)
    return total_loss/total_count, total_acc/total_count

In [ ]:
def fit(epochs, train_dl, test_dl):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []

    for epoch in range(epochs):
        epoch_loss, epoch_acc = train(train_dl)
        epoch_test_loss, epoch_test_acc = test(test_dl)
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc)
        test_loss.append(epoch_test_loss)
        test_acc.append(epoch_test_acc)
        exp_lr_scheduler.step()
        template = ("epoch:{:2d}, train_loss: {:.5f}, train_acc: {:.1f}% ," 
                    "test_loss: {:.5f}, test_acc: {:.1f}%")
        print(template.format(
              epoch, epoch_loss, epoch_acc*100, epoch_test_loss, epoch_test_acc*100))
    print("Done!")
    
    return train_loss, test_loss, train_acc, test_acc

In [ ]:
EPOCHS = 10

In [ ]:
train_loss, test_loss, train_acc, test_acc = fit(EPOCHS, train_dataloader, test_dataloader)

KeyboardInterrupt: 